In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
DATA_PATH = 'E:/Edureka_Python-Course/Data Sources/Day_11/'

In [3]:
d1 = pd.read_csv(DATA_PATH+'BX-Book-Ratings.csv', encoding='ISO-8859–1', low_memory=False)
d2= pd.read_csv(DATA_PATH+'BX-Books.csv'        , encoding='ISO-8859–1', low_memory=False)
d3= pd.read_csv(DATA_PATH+'BX-Users.csv'        , encoding='ISO-8859–1', low_memory=False)

>### The association can useful for title or publisher

In [4]:
#Pre Processing Steps
d1['user_id'] = d1['user_id'].astype('object')

merged_df = (
    d1.merge(
        d3, how='left', on='user_id', sort=True,
    )
).merge(d2,how='left', on='isbn', sort=True,)
#book title is not vital for association
merged_df.drop(['book_title','book_author', 'year_of_publication', 'publisher'], axis=1, inplace=True)
#Age and Locations values consist of very high number of nans
merged_df.drop(['Location', 'Age',], axis=1, inplace=True)
#merged_df.head().transpose()

In [5]:
isbn_le = LabelEncoder().fit(merged_df.isbn)
merged_df['isbn'] = isbn_le.transform(merged_df['isbn'])

merged_df['user_id'] = pd.to_numeric(merged_df['user_id'])

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  1048575 non-null  int64
 1   isbn     1048575 non-null  int32
 2   rating   1048575 non-null  int64
dtypes: int32(1), int64(2)
memory usage: 28.0 MB


In [6]:
df_final = merged_df

In [7]:
basket = (df_final
          .groupby(['user_id', 'isbn'])['rating']
          .sum().reset_index().fillna(0)
         .set_index('user_id'))
basket

,isbn,rating
user_id,,
2,62661,0
7,103304,0
8,14124,5
8,15936,0
8,31945,5
...,...,...
278854,147422,7
278854,173620,0
278854,184366,6


In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1,)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(rating),(isbn),0.378839,0.999999,0.378839,1.0,1.000001,3.613819e-07,inf
